Ensure `torch` and `torchvision` are installed. If `pytorch3d` is not installed, install it using the following cell:

In [ ]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("1.13.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

In [ ]:
import os
import torch
import matplotlib.pyplot as plt

# Util function for loading meshes
from pytorch3d.io import load_objs_as_meshes, load_obj

# Data structures and functions for rendering
from pytorch3d.structures import Meshes
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.vis.texture_vis import texturesuv_image_matplotlib
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras,
    PointLights,
    DirectionalLights,
    Materials,
    RasterizationSettings,
    MeshRenderer,
    MeshRasterizer,
    SoftPhongShader,
    TexturesUV,
    TexturesVertex,
    HardFlatShader
)

from pytorch3d.renderer import TexturesVertex
from pytorch3d.io import load_obj, save_obj
from pytorch3d.ops import sample_points_from_meshes

import numpy as np

# add path for demo utils functions
import sys
import os
sys.path.append(os.path.abspath(''))
from pytorch3d.loss import (
    chamfer_distance,
    mesh_edge_loss,
    mesh_laplacian_smoothing,
    mesh_normal_consistency,
)

In [ ]:
from pytorch3d.loss import (
    chamfer_distance,
    mesh_edge_loss,
    mesh_laplacian_smoothing,
    mesh_normal_consistency,
)

If using **Google Colab**, fetch the utils file for plotting image grids:

In [ ]:
!wget https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/plot_image_grid.py
from plot_image_grid import image_grid

OR if running **locally** uncomment and run the following cell:

In [ ]:
# from utils import image_grid

### 1. Load a mesh and texture file

Load an `.obj` file and its associated `.mtl` file and create a **Textures** and **Meshes** object.

**Meshes** is a unique datastructure provided in PyTorch3D for working with batches of meshes of different sizes.

**TexturesUV** is an auxiliary datastructure for storing vertex uv and texture maps for meshes.

**Meshes** has several class methods which are used throughout the rendering pipeline.

If running this notebook using **Google Colab**, run the following cell to fetch the mesh obj and texture files and save it at the path `data/cow_mesh`:
If running locally, the data is already available at the correct path.

In [ ]:
!mkdir -p data/cow_mesh
!wget -P data/cow_mesh https://dl.fbaipublicfiles.com/pytorch3d/data/cow_mesh/cow.obj
!wget -P data/cow_mesh https://dl.fbaipublicfiles.com/pytorch3d/data/cow_mesh/cow.mtl
!wget -P data/cow_mesh https://dl.fbaipublicfiles.com/pytorch3d/data/cow_mesh/cow_texture.png

In [ ]:
def get_cube():
  verts = np.array([
      [-1, -1, -1],
      [-1, -1,  1],
      [-1,  1, -1],
      [-1,  1,  1],
      [ 1, -1, -1],
      [ 1, -1,  1],
      [ 1,  1, -1],
      [ 1,  1,  1],
  ], dtype=np.float32)

  faces = np.array([
      [0, 1, 3, 2],
      [0, 2, 6, 4],
      [0, 4, 5, 1],
      [1, 5, 7, 3],
      [2, 3, 7, 6],
      [4, 6, 7, 5],
  ])

  return verts, faces

In [ ]:
def catmull_clark_subdivision(verts, faces):
  num_verts, num_faces = verts.shape[0], faces.shape[0]

  # face centroids
  face_verts = verts[faces]
  face_points = face_verts.mean(axis=1)

  # edge data
  # vertexIndices, adjacentFaceIndices
  edges = []
  for face_idx, face in enumerate(faces):
    for first, second in zip(face, face[1:]):

      prev_face_idx = None
      for edge_idx in range(len(edges)):
        if first in edges[edge_idx][:2] and second in edges[edge_idx][:2]:
          prev_face_idx = edges[edge_idx][-1]
          edges[edge_idx].append(face_idx)
          break

      if prev_face_idx is None:
        edges.append([first, second, face_idx])

    prev_face_idx = None
    for edge_idx in range(len(edges)):
      if face[-1] in edges[edge_idx][:2] and face[0] in edges[edge_idx][:2]:
        prev_face_idx = edges[edge_idx][-1]
        edges[edge_idx].append(face_idx)
        break

    if prev_face_idx is None:
      edges.append([face[-1], face[0], face_idx])

  edges = np.array(edges)

  edge_points = []
  for edge in edges:
    face_verts_mean = np.mean(face_points[edge[2:]], axis=0)
    edge_midpoints = np.mean(verts[edge[:2]], axis=0)
    edge_point = (edge_midpoints + face_verts_mean)/2
    edge_points.append(edge_point)
  edge_points = np.array(edge_points)

  vert_points = []
  for v_idx, vert in enumerate(verts):
    vert_faces = []
    vert_edges = []
    vert_edge_midpoints = []
    for face_idx, face in enumerate(faces):
      if v_idx in face:
        vert_faces.append(face_idx)
    for edge_idx, edge in enumerate(edges):
      if v_idx in edge[:2]:
        edge_midpoint = np.mean(verts[edge[:2]], axis=0)
        vert_edge_midpoints.append(edge_midpoint)
        vert_edges.append(edge_idx)

    F = np.mean(face_points[vert_faces], axis=0)
    R = np.mean(vert_edge_midpoints, axis=0)

    n = len(vert_faces)
    vert_point = (F + 2*R + (n - 3)*vert) / n
    vert_points.append(vert_point)


  # e_idx plus offset is edge point id
  # face_idx plus offset is face point id
  # connections
  res_faces = []
  for face_idx, face in enumerate(faces):
    fp_idx = face_idx + num_verts

    for e_idx, edge in enumerate(edges):
      if face[0] in edge[:2] and face[1] in edge[:2]:
        ep1 = e_idx + num_verts + num_faces
      elif face[1] in edge[:2] and face[2] in edge[:2]:
        ep2 = e_idx + num_verts + num_faces
      elif face[2] in edge[:2] and face[3] in edge[:2]:
        ep3 = e_idx + num_verts + num_faces
      elif face[3] in edge[:2] and face[0] in edge[:2]:
        ep4 = e_idx + num_verts + num_faces
    new_face1 = [face[0], ep1, fp_idx, ep4]
    new_face2 = [ep1, face[1], ep2, fp_idx]
    new_face3 = [fp_idx, ep2, face[2], ep3]
    new_face4 = [ep4, fp_idx, ep3, face[3]]
    res_faces.extend([new_face1, new_face2, new_face3, new_face4])


  res_verts = np.concatenate((vert_points, face_points, edge_points), axis=0)
  res_faces = np.array(res_faces)

  return res_verts, res_faces

In [ ]:
def quads_to_tris(faces):
  new_faces = []
  for face in faces:
    f1 = face[:-1]
    f2 = [face[-2], face[-1], face[0]]
    new_faces.append(f1)
    new_faces.append(f2)

  return np.array(new_faces)

In [ ]:
def catmull_clark_subdivision_pyt(verts, faces):
  num_verts, num_faces = verts.shape[0], faces.shape[0]

  # face centroids
  face_verts = verts[faces]
  face_points = face_verts.mean(axis=1)

  # edge data
  # vertexIndices, adjacentFaceIndices
  edges = []
  for face_idx, face in enumerate(faces):
    for first, second in zip(face, face[1:]):

      prev_face_idx = None
      for edge_idx in range(len(edges)):
        if first in edges[edge_idx][:2] and second in edges[edge_idx][:2]:
          prev_face_idx = edges[edge_idx][-1]
          edges[edge_idx].append(face_idx)
          break

      if prev_face_idx is None:
        edges.append([first, second, face_idx])

    prev_face_idx = None
    for edge_idx in range(len(edges)):
      if face[-1] in edges[edge_idx][:2] and face[0] in edges[edge_idx][:2]:
        prev_face_idx = edges[edge_idx][-1]
        edges[edge_idx].append(face_idx)
        break

    if prev_face_idx is None:
      edges.append([face[-1], face[0], face_idx])

  edges = torch.tensor(edges)

  edge_points = []
  for edge in edges:
    face_verts_mean = torch.mean(face_points[edge[2:]], axis=0)
    edge_midpoints = torch.mean(verts[edge[:2]], axis=0)
    edge_point = (edge_midpoints + face_verts_mean)/2
    edge_points.append(edge_point)
  edge_points = torch.stack(edge_points)

  vert_points = []
  for v_idx, vert in enumerate(verts):
    vert_faces = []
    vert_edges = []
    vert_edge_midpoints = []
    for face_idx, face in enumerate(faces):
      if v_idx in face:
        vert_faces.append(face_idx)
    for edge_idx, edge in enumerate(edges):
      if v_idx in edge[:2]:
        edge_midpoint = torch.mean(verts[edge[:2]], axis=0)
        vert_edge_midpoints.append(edge_midpoint)
        vert_edges.append(edge_idx)

    F = torch.mean(face_points[vert_faces], axis=0)
    R = torch.mean(torch.stack(vert_edge_midpoints), axis=0)

    n = len(vert_faces)
    vert_point = (F + 2*R + (n - 3)*vert) / n
    vert_points.append(vert_point)


  # e_idx plus offset is edge point id
  # face_idx plus offset is face point id
  # connections
  res_faces = []
  for face_idx, face in enumerate(faces):
    fp_idx = face_idx + num_verts

    for e_idx, edge in enumerate(edges):
      if face[0] in edge[:2] and face[1] in edge[:2]:
        ep1 = e_idx + num_verts + num_faces
      elif face[1] in edge[:2] and face[2] in edge[:2]:
        ep2 = e_idx + num_verts + num_faces
      elif face[2] in edge[:2] and face[3] in edge[:2]:
        ep3 = e_idx + num_verts + num_faces
      elif face[3] in edge[:2] and face[0] in edge[:2]:
        ep4 = e_idx + num_verts + num_faces
    new_face1 = [face[0], ep1, fp_idx, ep4]
    new_face2 = [ep1, face[1], ep2, fp_idx]
    new_face3 = [fp_idx, ep2, face[2], ep3]
    new_face4 = [ep4, fp_idx, ep3, face[3]]
    res_faces.extend([new_face1, new_face2, new_face3, new_face4])


  res_verts = torch.concatenate((torch.stack(vert_points), face_points, edge_points), axis=0)
  res_faces = torch.tensor(res_faces)

  return res_verts, res_faces

In [ ]:
def quads_to_tris_pyt(faces):
  new_faces = []
  for face in faces:
    f1 = face[:-1]
    f2 = torch.tensor([face[-2], face[-1], face[0]])
    new_faces.append(f1)
    new_faces.append(f2)

  return torch.stack(new_faces)

In [ ]:
verts, faces = get_cube()
verts = torch.tensor(verts)
faces = torch.tensor(faces)
verts, faces = catmull_clark_subdivision_pyt(verts, faces)
faces = quads_to_tris_pyt(faces)

In [ ]:

device = torch.device('cuda')

verts = torch.tensor(verts).to(device)
faces = torch.tensor(faces).to(device)

#final_obj = os.path.join('./', 'cube1.obj')
#save_obj(final_obj, verts, faces)

In [ ]:
verts_rgb = torch.ones_like(verts)[None] # (1, V, 3)
textures = TexturesVertex(verts_features=verts_rgb.to(device))
mesh = Meshes(verts=[verts], faces=[faces], textures=textures)

In [ ]:
gt_verts, gt_faces, gt_aux = load_obj('gt_mesh.obj')

gt_verts = gt_verts.to(device)
gt_faces = gt_faces.verts_idx.to(device)

gt_mesh = Meshes(verts=[gt_verts], faces=[gt_faces])


In [ ]:
fig2 = plot_scene({
    "plot1": {
        "gt_mesh": gt_mesh
    }
},
    xaxis={"backgroundcolor":"rgb(200, 200, 230)"},
    yaxis={"backgroundcolor":"rgb(230, 200, 200)"},
    zaxis={"backgroundcolor":"rgb(200, 230, 200)"},
    axis_args=AxisArgs(showgrid=True))
fig2.show()

In [ ]:
verts_cage, faces_cage = get_cube()
verts_subd, faces_subd = catmull_clark_subdivision(verts_cage, faces_cage)

verts = verts_subd
faces = faces_subd

verts = torch.tensor(verts).to(device)
faces = torch.tensor(quads_to_tris(faces)).to(device)


mesh = Meshes(verts=[verts], faces=[faces])



src_pts = sample_points_from_meshes(mesh, 1500)
gt_pts = sample_points_from_meshes(gt_mesh, 1500)
loss_chamfer, _ = chamfer_distance(src_pts, gt_pts)

In [ ]:
def get_cube_pyt():
  verts = torch.tensor([
      [-1, -1, -1],
      [-1, -1,  1],
      [-1,  1, -1],
      [-1,  1,  1],
      [ 1, -1, -1],
      [ 1, -1,  1],
      [ 1,  1, -1],
      [ 1,  1,  1],
  ], dtype=torch.float)

  faces = torch.tensor([
      [0, 1, 3, 2],
      [0, 2, 6, 4],
      [0, 4, 5, 1],
      [1, 5, 7, 3],
      [2, 3, 7, 6],
      [4, 6, 7, 5],
  ])

  return verts, faces

In [ ]:
v, f = get_cube_pyt()

In [ ]:
# run multiple times to restore multiple levels
verts_cage, faces_cage = v, f
verts_cage = torch.autograd.Variable(verts_cage.data, requires_grad=True)
optimizer = torch.optim.SGD([verts_cage], lr=0.15, momentum=0.9)

iter = range(100)
for i in iter:
  optimizer.zero_grad()

  v,f = verts_cage, faces_cage
  subd_count = 1
  for j in range(subd_count):
    v, f = catmull_clark_subdivision_pyt(v, f)


  mesh = Meshes(verts=[v], faces=[quads_to_tris_pyt(f)])


  pt_num = 15000
  src_pts = sample_points_from_meshes(mesh, pt_num).to(device)
  gt_pts = sample_points_from_meshes(gt_mesh, pt_num).to(device)


  loss_chamfer, _ = chamfer_distance(src_pts, gt_pts)
  loss_edge = mesh_edge_loss(mesh)
  loss_normal = mesh_normal_consistency(mesh)
  loss_laplacian = mesh_laplacian_smoothing(mesh, method="uniform")

  loss = loss_chamfer + loss_edge + loss_normal + loss_laplacian

  if i % 5 == 0:
    print(f'i {i} - {loss}')

  loss.backward(retain_graph=True)
  optimizer.step()

In [ ]:
print(verts_cage)

In [ ]:
v,f = verts_cage, faces_cage
subd_level = 1
for j in range(subd_level):
  v, f = catmull_clark_subdivision_pyt(v, f)

mesh = Meshes(verts=[v], faces=[quads_to_tris_pyt(f)])

fig2 = plot_scene({
    "cow_plot1": {
        "cows": mesh
    }
},
    xaxis={"backgroundcolor":"rgb(200, 200, 230)"},
    yaxis={"backgroundcolor":"rgb(230, 200, 200)"},
    zaxis={"backgroundcolor":"rgb(200, 230, 200)"},
    axis_args=AxisArgs(showgrid=True))
fig2.show()

In [ ]:
v,f = verts_cage, faces_cage
with open('export.obj', 'w') as file:
    file.write("# OBJ file\n")
    for vertex in v:
        file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")
    for face in f:
        file.write("f")
        for i in face:
            file.write(" %d" % (i + 1))
        file.write("\n")

In [ ]:
verts_cage, faces_cage = get_cube_pyt()
mesh = Meshes(verts=[verts_cage], faces=[quads_to_tris_pyt(faces_cage)])

fig2 = plot_scene({
    "cow_plot1": {
        "cows": gt_mesh
    }
},
    xaxis={"backgroundcolor":"rgb(200, 200, 230)"},
    yaxis={"backgroundcolor":"rgb(230, 200, 200)"},
    zaxis={"backgroundcolor":"rgb(200, 230, 200)"},
    axis_args=AxisArgs(showgrid=True))
fig2.show()

In [ ]:
import torch
import numpy as np


def get_cube_pyt():
  verts = torch.tensor([
      [-1, -1, -1],
      [-1, -1,  1],
      [-1,  1, -1],
      [-1,  1,  1],
      [ 1, -1, -1],
      [ 1, -1,  1],
      [ 1,  1, -1],
      [ 1,  1,  1],
  ], dtype=torch.float)

  faces = torch.tensor([
      [0, 1, 3, 2],
      [0, 2, 6, 4],
      [0, 4, 5, 1],
      [1, 5, 7, 3],
      [2, 3, 7, 6],
      [4, 6, 7, 5],
  ])

  return verts, faces

def catmull_clark_subdivision(verts, faces):
  num_verts, num_faces = verts.shape[0], faces.shape[0]

  # face centroids
  face_verts = verts[faces]
  face_points = face_verts.mean(axis=1)

  # edge data
  # vertexIndices, adjacentFaceIndices
  edges = []
  for face_idx, face in enumerate(faces):
    for first, second in zip(face, face[1:]):

      prev_face_idx = None
      for edge_idx in range(len(edges)):
        if first in edges[edge_idx][:2] and second in edges[edge_idx][:2]:
          prev_face_idx = edges[edge_idx][-1]
          edges[edge_idx].append(face_idx)
          break

      if prev_face_idx is None:
        edges.append([first, second, face_idx])

    prev_face_idx = None
    for edge_idx in range(len(edges)):
      if face[-1] in edges[edge_idx][:2] and face[0] in edges[edge_idx][:2]:
        prev_face_idx = edges[edge_idx][-1]
        edges[edge_idx].append(face_idx)
        break

    if prev_face_idx is None:
      edges.append([face[-1], face[0], face_idx])

  edges = np.array(edges)

  edge_points = []
  for edge in edges:
    face_verts_mean = np.mean(face_points[edge[2:]], axis=0)
    edge_midpoints = np.mean(verts[edge[:2]], axis=0)
    edge_point = (edge_midpoints + face_verts_mean)/2
    edge_points.append(edge_point)
  edge_points = np.array(edge_points)

  vert_points = []
  for v_idx, vert in enumerate(verts):
    vert_faces = []
    vert_edges = []
    vert_edge_midpoints = []
    for face_idx, face in enumerate(faces):
      if v_idx in face:
        vert_faces.append(face_idx)
    for edge_idx, edge in enumerate(edges):
      if v_idx in edge[:2]:
        edge_midpoint = np.mean(verts[edge[:2]], axis=0)
        vert_edge_midpoints.append(edge_midpoint)
        vert_edges.append(edge_idx)

    F = np.mean(face_points[vert_faces], axis=0)
    R = np.mean(vert_edge_midpoints, axis=0)

    n = len(vert_faces)
    vert_point = (F + 2*R + (n - 3)*vert) / n
    vert_points.append(vert_point)


  # e_idx plus offset is edge point id
  # face_idx plus offset is face point id
  # connections
  res_faces = []
  for face_idx, face in enumerate(faces):
    fp_idx = face_idx + num_verts

    for e_idx, edge in enumerate(edges):
      if face[0] in edge[:2] and face[1] in edge[:2]:
        ep1 = e_idx + num_verts + num_faces
      elif face[1] in edge[:2] and face[2] in edge[:2]:
        ep2 = e_idx + num_verts + num_faces
      elif face[2] in edge[:2] and face[3] in edge[:2]:
        ep3 = e_idx + num_verts + num_faces
      elif face[3] in edge[:2] and face[0] in edge[:2]:
        ep4 = e_idx + num_verts + num_faces
    new_face1 = [face[0], ep1, fp_idx, ep4]
    new_face2 = [ep1, face[1], ep2, fp_idx]
    new_face3 = [fp_idx, ep2, face[2], ep3]
    new_face4 = [ep4, fp_idx, ep3, face[3]]
    res_faces.extend([new_face1, new_face2, new_face3, new_face4])


  res_verts = np.concatenate((vert_points, face_points, edge_points), axis=0)
  res_faces = np.array(res_faces)

  return res_verts, res_faces

def catmull_clark_subdivision_pyt(verts, faces):
  num_verts, num_faces = verts.shape[0], faces.shape[0]

  # face centroids
  face_verts = verts[faces]
  face_points = face_verts.mean(dim=1)

  # edge data
  # vertexIndices, adjacentFaceIndices
  edges = []
  for face_idx, face in enumerate(faces):
    for first, second in zip(face, face[1:]):

      prev_face_idx = None
      for edge_idx in range(len(edges)):
        if first in edges[edge_idx][:2] and second in edges[edge_idx][:2]:
          prev_face_idx = edges[edge_idx][-1]
          edges[edge_idx].append(face_idx)
          break

      if prev_face_idx is None:
        edges.append([first, second, face_idx])

    prev_face_idx = None
    for edge_idx in range(len(edges)):
      if face[-1] in edges[edge_idx][:2] and face[0] in edges[edge_idx][:2]:
        prev_face_idx = edges[edge_idx][-1]
        edges[edge_idx].append(face_idx)
        break

    if prev_face_idx is None:
      edges.append([face[-1], face[0], face_idx])

  edges = torch.tensor(edges)

  edge_points = []
  for edge in edges:
    face_verts_mean = torch.mean(face_points[edge[2:]], axis=0)
    edge_midpoints = torch.mean(verts[edge[:2]], axis=0)
    edge_point = (edge_midpoints + face_verts_mean)/2
    edge_points.append(edge_point)
  edge_points = torch.stack(edge_points)

  vert_points = []
  for v_idx, vert in enumerate(verts):
    vert_faces = []
    vert_edges = []
    vert_edge_midpoints = []
    for face_idx, face in enumerate(faces):
      if v_idx in face:
        vert_faces.append(face_idx)
    for edge_idx, edge in enumerate(edges):
      if v_idx in edge[:2]:
        edge_midpoint = torch.mean(verts[edge[:2]], axis=0)
        vert_edge_midpoints.append(edge_midpoint)
        vert_edges.append(edge_idx)

    F = torch.mean(face_points[vert_faces], axis=0)
    R = torch.mean(torch.stack(vert_edge_midpoints), axis=0)

    n = len(vert_faces)
    vert_point = (F + 2*R + (n - 3)*vert) / n
    vert_points.append(vert_point)


  # e_idx plus offset is edge point id
  # face_idx plus offset is face point id
  # connections
  res_faces = []
  for face_idx, face in enumerate(faces):
    fp_idx = face_idx + num_verts

    for e_idx, edge in enumerate(edges):
      if face[0] in edge[:2] and face[1] in edge[:2]:
        ep1 = e_idx + num_verts + num_faces
      elif face[1] in edge[:2] and face[2] in edge[:2]:
        ep2 = e_idx + num_verts + num_faces
      elif face[2] in edge[:2] and face[3] in edge[:2]:
        ep3 = e_idx + num_verts + num_faces
      elif face[3] in edge[:2] and face[0] in edge[:2]:
        ep4 = e_idx + num_verts + num_faces
    new_face1 = [face[0], ep1, fp_idx, ep4]
    new_face2 = [ep1, face[1], ep2, fp_idx]
    new_face3 = [fp_idx, ep2, face[2], ep3]
    new_face4 = [ep4, fp_idx, ep3, face[3]]
    res_faces.extend([new_face1, new_face2, new_face3, new_face4])


  res_verts = torch.concatenate((torch.stack(vert_points), face_points, edge_points), axis=0)
  res_faces = torch.tensor(res_faces)

  return res_verts, res_faces

v, f = get_cube_pyt()
v1, f1 = catmull_clark_subdivision(v.numpy(),f.numpy())
#print(v1,f1)
v, f = catmull_clark_subdivision_pyt(v,f)
test1 = (v.numpy()==v1).all()
test2 = (f.numpy()==f1).all()
print(test1)
print(test2)


In [ ]:
import time
start_time = time.time()
v, f = get_cube_pyt()
for i in range(4):
  v, f = catmull_clark_subdivision_pyt(v,f)
print("--- %s seconds ---" % (time.time() - start_time))